In [73]:
import pandas as pd
import numpy as np

import datetime
import pickle

import os
import glob

from geospacepy import omnireader
from sunpy.net import hek, Fido, attrs as a
from sunpy.timeseries import TimeSeries


In [15]:
pd.__version__

'0.24.1'

In [16]:
np.__version__

'1.15.4'

In [17]:
def NewellCF_calc(v,bz,by):
    # v expected in km/s
    # b's expected in nT    
    NCF = np.zeros_like(v)
    NCF.fill(np.nan)
    bt = np.sqrt(by**2 + bz**2)
    bztemp = bz
    bztemp[bz == 0] = .001
    #Caculate clock angle (theta_c = t_c)
    tc = np.arctan2(by,bztemp)
    neg_tc = bt*np.cos(tc)*bz < 0 
    tc[neg_tc] = tc[neg_tc] + np.pi
    sintc = np.abs(np.sin(tc/2.))
    NCF = (v**1.33333)*(sintc**2.66667)*(bt**0.66667)
    return NCF

In [74]:
def download_omni_text(input_datetimes):



    #--------------------------------------------------------#
    #	OMNI Data - includes solar wind, and geomag params   #
    #--------------------------------------------------------#
    
    for d in range(len(input_datetimes)):
         
        loop_datetime = input_datetimes[d]
        print(loop_datetime)
        print(type(loop_datetime))
        t_start = loop_datetime - datetime.timedelta(1)
#         t_start = loop_datetime - np.timedelta64(1,'D')
        t_end = loop_datetime + datetime.timedelta(1) + datetime.timedelta(minutes = 10)
#         t_end = loop_datetime + np.timedelta64(1,'D') + np.timedelta64(10,'m')


        t_start_day = loop_datetime
        t_end_day = loop_datetime + datetime.timedelta(minutes = 1439)
#         t_end_day = loop_datetime + np.timedelta64(1439,'m')

        
        #get OMNI data
        omniInt = omnireader.omni_interval(t_start,t_end,'5min', cdf_or_txt = 'txt')

        #print(omniInt.cdfs[0].vars) #prints all the variables available on omni

        epochs = omniInt['Epoch'] #time array for omni 5min data
        By,Bz,AE,SymH = omniInt['BY_GSM'],omniInt['BZ_GSM'],omniInt['AE_INDEX'], omniInt['SYM_H']
        vsw,psw = omniInt['flow_speed'], omniInt['Pressure']
        borovsky_reader = omnireader.borovsky(omniInt)
        borovsky = borovsky_reader()
        #newell_reader = omnireader.newell(omniInt)
        #newell = newell_reader()



        newell = NewellCF_calc(vsw, Bz, By)


        proton_flux_10MeV, proton_flux_30MeV, proton_flux_60MeV = omniInt['PR-FLX_10'], omniInt['PR-FLX_30'], omniInt['PR-FLX_60']





        #calculate clock angle
        clock_angle = np.degrees(np.arctan2( By,Bz))
        clock_angle[clock_angle < 0] = clock_angle[clock_angle<0] + 360.

        print('Got 5 minutes data')

        omniInt_1hr = omnireader.omni_interval(t_start,t_end,'hourly', cdf_or_txt = 'txt')
        epochs_1hr = omniInt_1hr['Epoch'] #datetime timestamps
        F107,KP = omniInt_1hr['F10_INDEX'],omniInt_1hr['KP']


        print('Got hour data')
        #--------------------------------------------------------#
        #	GOES X-ray data - Channel 1-8A, defines flare class  #
        #--------------------------------------------------------#


        results = Fido.search(a.Time(t_start, t_end), a.Instrument('XRS'))
        files = Fido.fetch(results)
        goes = TimeSeries(files, concatenate = True)

        goes_l = goes.data['xrsb']

        print('Got GOES data')
        #--------------------------------------------------------#
        #	Resample data to 1min to match GNSS CHAIN network    #
        #--------------------------------------------------------#


        #resample OMNI Solar Wind Data
        By_data = pd.Series(By, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        Bz_data = pd.Series(Bz, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        AE_data = pd.Series(AE, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        SymH_data = pd.Series(SymH, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        vsw_data = pd.Series(vsw, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        psw_data = pd.Series(psw, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        borovsky_data = pd.Series(borovsky, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        newell_data = pd.Series(newell, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        proton_10_data = pd.Series(proton_flux_10MeV, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        proton_30_data = pd.Series(proton_flux_30MeV, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        proton_60_data = pd.Series(proton_flux_60MeV, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)
        clock_angle_data = pd.Series(clock_angle, index = epochs).resample('1T').pad().truncate(t_start_day, t_end_day)

        F107data = pd.Series(F107, index = epochs_1hr).resample('1T').pad().truncate(t_start_day, t_end_day)
        KPdata = pd.Series(KP, index = epochs_1hr).resample('1T').pad().truncate(t_start_day, t_end_day)


        #function to find data at previous time intervals
        def roll_back(data, minutes = 1):
            ts = t_start_day - datetime.timedelta(minutes = minutes)
            te = t_end_day - datetime.timedelta(minutes = minutes)
#             ts = t_start_day - np.timedelta64(minutes,'m')
#             te = t_end_day - np.timedelta64(minutes,'m')
            data = pd.Series(data, index = epochs).resample('1T').pad()
            new_data = data.truncate(ts, te)
            rolled_data = pd.Series(np.array(new_data), index = By_data.index)
            return rolled_data

        #calculate rolled back timeseries - 15 and 30 minutes previous
        By_15 = roll_back(By, minutes = 15)
        By_30 = roll_back(By, minutes = 30)
        Bz_15 = roll_back(Bz, minutes = 15)
        Bz_30 = roll_back(Bz, minutes = 30)
        AE_15 = roll_back(AE, minutes = 15)
        AE_30 = roll_back(AE, minutes = 30)
        SymH_15 = roll_back(SymH, minutes = 15)
        SymH_30 = roll_back(SymH, minutes = 30)
        vsw_15 = roll_back(vsw, minutes = 15)
        vsw_30 = roll_back(vsw, minutes = 30)
        psw_15 = roll_back(psw, minutes = 15)
        psw_30 = roll_back(psw, minutes = 30)	
        borovsky_15 = roll_back(borovsky, minutes = 15)
        borovsky_30 = roll_back(borovsky, minutes = 30)
        newell_15 = roll_back(newell, minutes = 15)
        newell_30 = roll_back(newell, minutes = 30)
        clock_angle_15 = roll_back(clock_angle, minutes = 15)
        clock_angle_30 = roll_back(clock_angle, minutes = 30)
        #resample GOES X-ray flux
        goes_data = goes_l.resample('1T').mean().truncate(t_start_day, t_end_day)

        #put all in a dataframe and save

        dataframe = pd.DataFrame()
        dataframe['Bz - 0min [nT]'] = Bz_data
        dataframe['Bz - 15min [nT]'] = Bz_15
        dataframe['Bz - 30min [nT]'] = Bz_30

        dataframe['By - 0min [nT]'] = By_data
        dataframe['By - 15min [nT]'] = By_15
        dataframe['By - 30min [nT]'] = By_30

        dataframe['Vsw - 0min [km/s]'] = vsw_data
        dataframe['Vsw - 15min [km/s]'] = vsw_15
        dataframe['Vsw - 30min [km/s]'] = vsw_30

        dataframe['Psw - 0min [nPa]'] = psw_data
        dataframe['Psw - 15min [nPa]'] = psw_15
        dataframe['Psw - 30min [nPa]'] = psw_30

        dataframe['AE - 0min [nT]'] = AE_data
        dataframe['AE - 15min [nT]'] = AE_15
        dataframe['AE - 30min [nT]'] = AE_30

        dataframe['SymH - 0min [nT]'] = SymH_data
        dataframe['SymH - 15min [nT]'] = SymH_15
        dataframe['SymH - 30min [nT]'] = SymH_30


        dataframe['Clock Angle - 0min [deg]'] = clock_angle_data
        dataframe['Clock Angle - 15min [deg]'] = clock_angle_15
        dataframe['Clock Angle - 30min [deg]'] = clock_angle_30

        dataframe['Newell CF - 0min [m/s^(4/3) T^(2/3)]'] = newell_data
        dataframe['Newell CF - 15min [m/s^(4/3) T^(2/3)]'] = newell_15
        dataframe['Newell CF - 30min [m/s^(4/3) T^(2/3)]'] = newell_30

        dataframe['Borovsky CF - 0min [nT km/s]'] = borovsky_data
        dataframe['Borovsky CF - 15min [nT km/s]'] = borovsky_15
        dataframe['Borovsky CF - 30min [nT km/s]'] = borovsky_30

        dataframe['Kp [dimensionless]'] = KPdata
        dataframe['F107 [sfu=10^-22 W/m^2/hz]'] = F107data


        dataframe['Proton 10MeV'] = proton_10_data
        dataframe['Proton 30MeV'] = proton_30_data
        dataframe['Proton 60MeV'] = proton_60_data

        dataframe['GOES X-ray Wm^-2'] = goes_data
        dataframe_nan = dataframe.replace(9999.99, np.nan) #replace 9999.99 with nans

        filepath = '/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/'
        filename = filepath + 'solardata' + loop_datetime.strftime('%Y') + '_' +loop_datetime.strftime('%j') + '.csv'
        print('output solardata file location = {}'.format(filename))

        dataframe_nan.to_csv(filename, index_label = 'Datetime')


In [19]:
directory = "/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/"


for file in glob.glob(os.path.join(directory,"*ions.pkl")):
    print(file)
    
    # get all datetimes


/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_6936_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_6937_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_7063_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_1459_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_1458_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_8268_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_8269_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_1482_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_1483_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_1425_ions.pkl
/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_94

In [20]:
# file = "/Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/132_cleaned_orbits/orb_6936_ions.pkl"
file = "/Users/ryanmcgranaghan/Downloads/fast_orbits.pkl"
data_in = open(file,'rb')
data = pickle.load(data_in)
print(data)

                              BMagNEC      B_NGeod       B_UGeod          alt  \
1996-12-30 20:11:52.625  13781.766280   856.946733 -13739.723308  4067.885145   
1996-12-30 20:11:52.750  13782.558003   856.897347 -13740.510483  4067.936690   
1996-12-30 20:11:52.875  13781.715776   857.576196 -13739.623624  4067.988235   
1996-12-30 20:11:53.000  13780.606869   857.076499 -13738.566206  4068.039780   
1996-12-30 20:11:53.125  13780.949598   856.795642 -13738.901338  4068.092301   
1996-12-30 20:11:53.250  13780.984137   856.624726 -13738.948145  4068.143845   
1996-12-30 20:11:53.375  13781.718723   856.981653 -13739.685753  4068.195390   
1996-12-30 20:11:53.500  13781.648755   857.795216 -13739.571130  4068.246935   
1996-12-30 20:11:53.625  13781.003494   857.727643 -13738.960334  4068.299455   
1996-12-30 20:11:53.750  13780.761654   857.937878 -13738.687645  4068.350998   
1996-12-30 20:11:53.875  13781.460698   857.936007 -13739.375061  4068.402542   
1996-12-30 20:11:54.000  137

In [21]:
data = pd.read_pickle(file)

In [22]:
data.info()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1133461 entries, 1996-12-30 20:11:52.625000 to 1999-01-15 16:45:56.625000
Data columns (total 26 columns):
BMagNEC         1133461 non-null float64
B_NGeod         1133461 non-null float64
B_UGeod         1133461 non-null float64
alt             1133461 non-null float64
dBe1            1133461 non-null float64
dBe2            1133461 non-null float64
dBpar           1133461 non-null float64
gdlat           1133461 non-null float64
ilat            1133461 non-null float64
mapratio        1133461 non-null float32
mlon            1133461 non-null float64
mlt             1133461 non-null float64
orbit           1133461 non-null float64
parvelocity     1133461 non-null float64
perpvelocity    1133461 non-null float64
scspeed         1133461 non-null float64
j               9716 non-null float64
je              9716 non-null float64
jeerr           9716 non-null float64
jerr            9716 non-null float64
n               9716 non-null fl

In [61]:
dts  = pd.to_datetime(data.index.values)


In [75]:
download_omni_text(dts)



1996-12-30 20:11:52.625000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:52.750000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data


Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:54.375000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospacePar

[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:56
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:56.125000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, sta

Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:57.750000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        =====================================

[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:59.375000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:11:59.500000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5m

...

Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:13:22
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got

[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:13:23.625000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:13:23.750000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5m

1996-12-30 20:13:25.250000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data
[                                        ========================================
Got GOES data
output solardata file location = /Users/ryanmcgranaghan/Documents/Conferences/ISSI_2018/ISSI_geospaceParticles/solar_data/solardata1996_365.csv
1996-12-30 20:13:25.375000
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Created interval between 1996-12-29 and 1996-12-31, cadence 5min, start index 104787, end index 105365
Got 5 minutes data
Created interval between 1996-12-29 and 1996-12-31, cadence hourly, start index 8733, end index 8781
Applying transform Hourly Kp*10 -> Kp to omni hourly variable KP
Got hour data


Exception in thread Thread-2270:
Traceback (most recent call last):
  File "/anaconda3/envs/python36/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/anaconda3/envs/python36/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/a

KeyboardInterrupt: 

In [35]:
input_datetime = dts.values[0]

numpy.datetime64('1996-12-30T20:11:52.624999999')

In [47]:
chc = datetime.datetime(1996,12,30,20,11,52)
print(chc)
print(type(chc))
print(chc - datetime.timedelta(1))

1996-12-30 20:11:52
<class 'datetime.datetime'>
1996-12-29 20:11:52


In [51]:
print(input_datetime)
print(type(input_datetime))

print(input_datetime - np.timedelta64(1,'D'))

1996-12-30T20:11:52.625000000
<class 'numpy.datetime64'>
1996-12-29T20:11:52.625000000


In [56]:
print(input_datetime + np.timedelta64(10,'m'))

1996-12-30T20:21:52.625000000
